## Dataset Construction

I constructed a plan–county–year dataset for Medicare Advantage plans in 2018 by combining monthly enrollment data with contract-level information. Monthly data were collapsed to average enrollment by plan and county and restricted to approved counties using Medicare Advantage service area files.


In [1]:
# MERGE TO BUILD THE 2018 PLAN–COUNTY–YEAR DATASET
# I combine 12 months of enrollment + contract info, then collapse to plan–county–year, then inner merge with service area counties.

import pandas as pd
from pathlib import Path

YEAR = 2018
MONTHS = [f"{m:02d}" for m in range(1, 13)]

DATA_DIR = Path("data")
CONTRACT_DIR = DATA_DIR / "ma_enrollment_contract_2018"
ENROLL_DIR   = DATA_DIR / "ma_enrollment_plan_2018"
SERVICE_DIR  = DATA_DIR / "ma_service_data_2018"

CONTRACT_COLUMNS = [
    "contractid", "planid", "org_type", "plan_type", "partd", "snp", "eghp",
    "org_name", "org_marketing_name", "plan_name", "parent_org", "contract_date",
]
ENROLL_COLUMNS = ["contractid", "planid", "ssa", "fips", "state", "county", "enrollment"]

monthly_list = []
for m in MONTHS:
    contract_path = CONTRACT_DIR / f"CPSC_Contract_Info_{YEAR}_{m}.csv"
    enroll_path   = ENROLL_DIR   / f"CPSC_Enrollment_Info_{YEAR}_{m}.csv"

    contract = pd.read_csv(contract_path, low_memory=False, encoding="latin1")
    contract = contract.rename(columns={
        "Contract ID": "contractid",
        "Plan ID": "planid",
        "Organization Type": "org_type",
        "Plan Type": "plan_type",
        "Offers Part D": "partd",
        "SNP Plan": "snp",
        "EGHP": "eghp",
        "Organization Name": "org_name",
        "Organization Marketing Name": "org_marketing_name",
        "Plan Name": "plan_name",
        "Parent Organization": "parent_org",
        "Contract Date": "contract_date",
    })
    contract["contractid"] = contract["contractid"].astype("string")
    contract["planid"] = pd.to_numeric(contract["planid"], errors="coerce")
    contract = contract.drop_duplicates(subset=["contractid", "planid"])
    contract = contract[[c for c in CONTRACT_COLUMNS if c in contract.columns]]

    enroll = pd.read_csv(enroll_path, na_values=["*"], low_memory=False, encoding="latin1")
    enroll = enroll.rename(columns={
        "Contract Number": "contractid",
        "Plan ID": "planid",
        "SSA State County Code": "ssa",
        "FIPS State County Code": "fips",
        "State": "state",
        "County": "county",
        "Enrollment": "enrollment",
    })
    enroll["contractid"] = enroll["contractid"].astype("string")
    enroll["planid"] = pd.to_numeric(enroll["planid"], errors="coerce")
    enroll["enrollment"] = pd.to_numeric(enroll["enrollment"], errors="coerce")
    enroll = enroll[[c for c in ENROLL_COLUMNS if c in enroll.columns]]

    df = pd.merge(contract, enroll, on=["contractid", "planid"], how="left")
    df["month"] = int(m)
    df["year"] = YEAR

    df["fips5"] = (
        pd.to_numeric(df["fips"], errors="coerce")
          .round()
          .astype("Int64")
          .astype("string")
          .str.zfill(5)
    )

    monthly_list.append(df)

monthly = pd.concat(monthly_list, ignore_index=True)

# ✅ CHECK (remove later)
print("CHECK: monthly stacked rows/cols =", monthly.shape)

plan_county_year = (
    monthly
    .groupby(["contractid", "planid", "fips5", "state", "county", "year"], as_index=False)
    .agg(
        avg_enrollment=("enrollment", "mean"),  
        plan_type=("plan_type", "last"),
        snp=("snp", "last"),
        eghp=("eghp", "last"),
    )
)

# ✅ CHECK (remove later)
print("CHECK: collapsed plan_county_year rows/cols =", plan_county_year.shape)

service_list = []
for m in MONTHS:
    service_path = SERVICE_DIR / f"MA_Cnty_SA_{YEAR}_{m}.csv"
    sa = pd.read_csv(service_path, low_memory=False, encoding="latin1")
    service_list.append(sa)

service = pd.concat(service_list, ignore_index=True)

fips_candidates = [c for c in service.columns if "fips" in c.lower()]
if len(fips_candidates) == 0:
    raise ValueError("No FIPS column found in service area files.")
sa_fips_col = fips_candidates[0]

service["fips5"] = (
    pd.to_numeric(service[sa_fips_col], errors="coerce")
      .round()
      .astype("Int64")
      .astype("string")
      .str.zfill(5)
)

merge_keys = ["fips5"]

service = service.drop_duplicates(subset=merge_keys)

merged = pd.merge(plan_county_year, service, on=merge_keys, how="inner")

# Checking that merge worked
print("CHECK: service area columns =", list(service.columns))
print("CHECK: merge keys used =", merge_keys)
print("CHECK: merged dataset rows/cols =", merged.shape)




CHECK: monthly stacked rows/cols = (27710394, 19)
CHECK: collapsed plan_county_year rows/cols = (2477563, 10)
CHECK: service area columns = ['Contract ID', 'Organization Name', 'Organization Type', 'Plan Type', 'Partial', 'EGHP', 'SSA', 'FIPS', 'County', 'State', 'Notes', 'fips5']
CHECK: merge keys used = ['fips5']
CHECK: merged dataset rows/cols = (2474873, 21)


## Question 1: Plan Count by Plan Type


In [3]:
# QUESTION 1 - Provide a table of the count of plans under each plan type."

plans = merged[["contractid", "planid", "plan_type"]].drop_duplicates()

table1 = (
    plans
    .groupby("plan_type")
    .size()
    .reset_index(name=str(YEAR))
    .sort_values("plan_type")
)

table1


,plan_type,2018
0,1876 Cost,101
1,Employer/Union Only Direct Contract PDP,3
2,HMO/HMOPOS,2678
3,Local PPO,966
4,MSA,5
5,Medicare Prescription Drug Plan,1011
6,Medicare-Medicaid Plan HMO/HMOPOS,54
7,National PACE,258
8,PFFS,50
9,Regional PPO,109


## Question 2: Plan Count After Exclusions


In [4]:
# QUESTION 2 - "Remove Special Needs Plans (SNP), Employer Group Plans (EGHP), and 800-series plans. Provide updated Table 1."


# Identify SNP and EGHP plans
snp_true = merged["snp"].astype("string").str.upper().str.strip().isin(["Y", "YES", "1", "TRUE", "T"])
eghp_true = merged["eghp"].astype("string").str.upper().str.strip().isin(["Y", "YES", "1", "TRUE", "T"])

# Identify 800-series plan IDs (800–899)
planid_num = pd.to_numeric(merged["planid"], errors="coerce")
plan_800 = planid_num.between(800, 899)

# Apply exclusions
merged_excl = merged[~snp_true & ~eghp_true & ~plan_800].copy()

# Recompute plan counts by type
plans_excl = merged_excl[["contractid", "planid", "plan_type"]].drop_duplicates()

table2 = (
    plans_excl
    .groupby("plan_type")
    .size()
    .reset_index(name=str(YEAR))
    .sort_values("plan_type")
)

table2

,plan_type,2018
0,1876 Cost,93
1,HMO/HMOPOS,1569
2,Local PPO,569
3,MSA,3
4,Medicare Prescription Drug Plan,794
5,Medicare-Medicaid Plan HMO/HMOPOS,54
6,National PACE,258
7,PFFS,46
8,Regional PPO,49


## Question 3: Average Enrollment by Plan Type


In [5]:
# QUESTION 3 - Repeat filters from part 2 and focus only on approved counties, then provide average enrollments by plan type. 
# I.e. the service area data is data about approved counties.

table3 = (
    merged_excl
    .groupby("plan_type")["avg_enrollment"]
    .mean()
    .reset_index(name="avg_enrollment")
    .sort_values("plan_type")
)

table3

,plan_type,avg_enrollment
0,1876 Cost,249.314594
1,HMO/HMOPOS,737.999117
2,Local PPO,330.051222
3,MSA,58.131918
4,Medicare Prescription Drug Plan,335.196077
5,Medicare-Medicaid Plan HMO/HMOPOS,734.213457
6,National PACE,139.768954
7,PFFS,93.509485
8,Regional PPO,188.223442
